In [1]:
import pandas as pd
import numpy as np
import utils_local

from scipy import stats

from surprise import Dataset, Reader
from surprise import KNNBasic, KNNWithMeans
from surprise.model_selection import train_test_split

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from sklearn.metrics import root_mean_squared_error, mean_squared_error, mean_absolute_error, f1_score

In [2]:
# Genera and sites
path = "../data/AllSites_SiteOccurrences_AllGenera_26.1.24.csv"

df = pd.read_csv(path)
df.head()

,SITE_NAME,Equus,Coelodonta,Bos,Gazella,Ursus,Vulpes,Cervus,Canis,Sus,...,Total_Gen_Count,Large_GenCount,Small_GenCount,smallperlarge,smallprop,Herb_GenCount,Nonherb_GenCount,DietRatio,HerbProp,mid_age
0,Aba Zawei,1,1,1,1,0,0,0,0,0,...,4,4,0,0.0,0.000000,4,0,NaN,1.000000,0.0265
1,Abric Romani,1,0,1,0,1,1,1,1,1,...,12,12,0,0.0,0.000000,6,5,1.2,0.500000,0.0550
2,Acheng_Jiaojie,0,0,0,0,0,0,1,0,0,...,7,5,2,0.4,0.285714,5,2,2.5,0.714286,0.2100
3,Adler cave,1,0,0,0,0,1,0,1,0,...,10,5,5,1.0,0.500000,6,4,1.5,0.600000,0.0275
4,Adyrgan,1,0,0,1,0,0,0,0,0,...,11,5,6,1.2,0.545455,11,0,NaN,1.000000,2.2000


In [3]:
# Dental traits genera
path = "../data/DentalTraits_Genus_PPPA.csv"

df_dental = pd.read_csv(path, sep=",")
df_dental.head()

,Genus,n,Mass.g_Mean,Diet.Plant_Mean,HYP_Mean,LOP_Mean,FCT_AL_Mean,FCT_OL_Mean,FCT_SF_Mean,BUN_Mean,...,HYP_Mode,LOP_Mode,FCT_AL_Mode,FCT_OL_Mode,FCT_SF_Mode,BUN_Mode,Order,ConsInGenhyp,ConsInGen_bun,Unnamed: 21
0,Addax,1.0,70000.300000,100.0,3.0,2.0,0.0,1.0,1.0,0.0,...,3,2,0,1,1.0,0,Cetartiodactyla,True,True,NaN
1,Aepyceros,1.0,52500.100000,100.0,3.0,2.0,0.0,1.0,0.0,0.0,...,3,2,0,1,0.0,0,Cetartiodactyla,True,True,NaN
2,Alcelaphus,1.0,171001.500000,100.0,3.0,2.0,0.0,1.0,0.0,0.0,...,3,2,0,1,0.0,0,Cetartiodactyla,True,True,NaN
3,Alces,1.0,356998.000000,100.0,1.0,2.0,1.0,1.0,0.0,0.0,...,1,2,1,1,0.0,0,Cetartiodactyla,True,True,NaN
4,Allochrocebus,3.0,5708.333333,90.0,1.0,0.0,0.0,0.0,0.0,1.0,...,1,0,0,0,0.0,1,Primates,True,True,NaN


In [4]:
# Mass-diet
path = "../data/FossilGenera_MammalMassDiet_Jan24.csv"

df_mass_diet = pd.read_csv(path, sep=",")
df_mass_diet.head()

,Genus,Order,Family,MassSource,Massg,LogMass,LargeSmall,SizeClass,Diet,DietSource
0,Abudhabia,Rodentia,Muridae,Family average,1.343147e+02,2.128124,Small,small,Herbivore,Phylacine
1,Aceratherium,Perissodactyla,Rhinocerotidae,Cooke,1.099006e+06,6.041000,Large,large,Herbivore,Phylacine
2,Acinonyx,Carnivora,Felidae,Phylacine,4.670000e+04,4.669317,Large,large,Non-Herbivore,Phylacine
3,Aepyosciurus,Rodentia,Sciuridae,NOW,2.860000e+02,2.456366,Small,small,Herbivore,Phylacine
4,Aeretes,Rodentia,Sciuridae,Phylacine,7.324000e+02,2.864748,Small,small,Herbivore,Phylacine


In [5]:
path = "../data/DentalTraits_Genus_PPPA_ds.csv"

df_dental_new = pd.read_csv(path, sep=",")
df_dental_new.head()

,Genus,n,massg,HY,LOP,AL,OL,SF,BUN,OT,Excl_AL,ConsInGenhyp
0,Addax,1.0,70000.300000,3,2,0,1,1.0,0,0.0,0,True
1,Aepyceros,1.0,52500.100000,3,2,0,1,0.0,0,0.0,0,True
2,Alcelaphus,1.0,171001.500000,3,2,0,1,0.0,0,1.0,0,True
3,Alces,1.0,356998.000000,1,2,1,0,0.0,0,0.0,1,True
4,Allochrocebus,3.0,5708.333333,1,0,0,0,0.0,1,0.0,0,True


In [6]:
def build_site_genus_matrix(df):
    cols_redundant = ['LAT',
    'LONG',
    'ALTITUDE',
    'MAX_AGE',
    'BFA_MAX',
    'BFA_MAX_ABS',
    'MIN_AGE',
    'BFA_MIN',
    'BFA_MIN_ABS',
    'COUNTRY',
    'age_range',
    'Total_Gen_Count',
    'Large_GenCount',
    'Small_GenCount',
    'smallperlarge',
    'smallprop',
    'Herb_GenCount',
    'Nonherb_GenCount',
    'DietRatio',
    'HerbProp',
    'mid_age'
    ]


    df_site_genus = df.drop(columns=cols_redundant).set_index('SITE_NAME')

    return df_site_genus

## Building collaborative filtering

In [10]:
def fit_knn_collaborative_filtering(df, site_based=True, similarity="cosine", k=5, min_k=1):
    site_genus_matrix = build_site_genus_matrix(df)
    df_site_genus = site_genus_matrix.stack().reset_index().rename(columns={"level_1": "genus", 0: "presence"})

    reader = Reader(rating_scale=(0, 1))
    data = Dataset.load_from_df(df_site_genus, reader) # Column order must be user, item, rating

    sim_options = {
    'name': similarity,
    'user_based': site_based  # True for user-user, False for item-item
    }

    trainset = data.build_full_trainset()

    knn = KNNBasic(k=k, min_k=min_k, sim_options=sim_options)
    knn.fit(trainset)

    return knn, trainset, df_site_genus

In [11]:
knn, trainset, train_data = fit_knn_collaborative_filtering(
    df, 
    site_based=True, 
    similarity="cosine",
    k=5,
    min_k=1
)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [9]:
def predict_collaborative_filtering(algotihm, trainset):
    # Testset equal to trainset
    testset = trainset.build_testset()

    # Get predictions for all user-item pairs
    predictions = algotihm.test(testset)

    # Get item scores from the predictions
    item_scores = [(prediction.uid, prediction.iid, prediction.est) for prediction in predictions]
    return pd.DataFrame(item_scores, columns =['SITE_NAME', 'PREDICTED_GENUS', 'SCORE'])

## Hybrid algorithm content-knn